In [ ]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker

print("Fetching Schaefer atlas...")
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=100, yeo_networks=7, resolution_mm=2)
atlas_img = atlas.maps
labels = atlas.labels

In [3]:
import os
import numpy as np
from nilearn.input_data import NiftiLabelsMasker

def corr(fmri_path):
    try:
        masker = NiftiLabelsMasker(
            labels_img=atlas_img,
            standardize=True,
            detrend=True,
            memory='nilearn_cache',
            verbose=0
        )
        time_series = masker.fit_transform(fmri_path)
        correlation_matrix = np.corrcoef(time_series.T)

        base = os.path.basename(fmri_path).replace('.nii.gz', '')
        output_dir = "correlation_matrices"
        os.makedirs(output_dir, exist_ok=True)
        out_filename = os.path.join(output_dir, f"{base}_corr.npy")

        np.save(out_filename, correlation_matrix)
        print(f"Saved: {out_filename}")

    except Exception as e:
        print(f"Error processing {fmri_path}: {e}")


In [ ]:
def corr1(fmri_path):
    try:
        masker = NiftiLabelsMasker(
            labels_img=atlas_img,
            standardize=True,
            detrend=True,
            memory="nilearn_cache",
            verbose=0,
        )
        time_series = masker.fit_transform(fmri_path)
        corrs = []
        for i in network_to_indices:
            corrs.append(np.corrcoef(time_series[:, network_to_indices[i]].T))

        base = os.path.basename(fmri_path).replace(".nii.gz", "")
        output_dir = "subnet_matrices"
        os.makedirs(output_dir, exist_ok=True)
        out_filename = os.path.join(output_dir, f"{base}_corr.npz")

        np.savez(out_filename, **{f"arr_{i}": arr for i, arr in enumerate(corrs)})
        print(f"Saved: {out_filename}")

    except Exception as e:
        print(f"Error processing {fmri_path}: {e}")

In [4]:
import urllib.request as request
s3_prefix = 'https://s3.amazonaws.com/fcp-indi/data/Projects/'\
                'ABIDE_Initiative'
s3_pheno_path = '/'.join([s3_prefix, 'Phenotypic_V1_0b_preprocessed1.csv'])
s3_pheno_file = request.urlopen(s3_pheno_path)
pheno_list = s3_pheno_file.readlines()


In [5]:
header = pheno_list[0].decode().split(',')
file_idx = header.index('FILE_ID')

In [6]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import os
import urllib.request as request

def process_row(pheno_row):
    try:
        cs_row = pheno_row.decode().split(',')
        row_file_id = cs_row[file_idx]
        filename = f"{row_file_id}_func_preproc.nii.gz"
        s3_path = '/'.join([s3_prefix, 'Outputs', "cpac", "nofilt_noglobal", "func_preproc", filename])
        
        rel_path = s3_path.lstrip(s3_prefix)
        download_file = os.path.join(".", rel_path)
        download_dir = os.path.dirname(download_file)


        os.makedirs(download_dir, exist_ok=True)

        if not os.path.exists(download_file):
            request.urlretrieve(s3_path, download_file)
        corr(download_file)
        os.remove(download_file)

    except Exception as e:
        print(f"Error: {e}")

with ThreadPoolExecutor(max_workers=4) as executor:
    list(tqdm(executor.map(process_row, pheno_list[1:]), total=len(pheno_list[1:]), desc="Parallel Processing"))


Parallel Processing:   0%|          | 1/1112 [00:00<05:11,  3.57it/s]

Error: HTTP Error 404: Not Found


Parallel Processing:   0%|          | 2/1112 [00:53<9:38:45, 31.28s/it]

Saved: correlation_matrices/Pitt_0050003_func_preproc_corr.npy
Saved: correlation_matrices/Pitt_0050005_func_preproc_corr.npy


Parallel Processing:   0%|          | 3/1112 [00:54<5:20:45, 17.35s/it]

...

Saved: correlation_matrices/SBL_0051585_func_preproc_corr.npy


Parallel Processing: 100%|█████████▉| 1111/1112 [3:42:01<00:07,  7.85s/it]

Saved: correlation_matrices/MaxMun_a_0051606_func_preproc_corr.npy


Parallel Processing: 100%|██████████| 1112/1112 [3:42:02<00:00, 11.98s/it]

Saved: correlation_matrices/MaxMun_a_0051607_func_preproc_corr.npy
